In [25]:
if 'g' in locals():
    driver = g.driver
    g = CoinGame(driver=driver)
else:
    g = CoinGame()
    driver = g.driver

In [23]:
g.animation_wait_time = 0
for _ in range(1):
    g.flip_one_coin()
    data = g.get_data()
    print(data)

{'heads': 1, 'tails': 0, 'score': 0, 'flips_left': 99}
